# Component Test: Convert Model to ONNX

## Author
- Sebastian Lehrig <sebastian.lehrig1@ibm.com>

## License
Apache-2.0 License

## Imports & Constants

In [ ]:
import kfp
from kfp.components import OutputPath
import kfp.dsl as dsl

%load_ext lab_black

In [ ]:
BASE_IMAGE = "quay.io/ibm/kubeflow-notebook-image-ppc64le:latest"

KFP_CLIENT = kfp.Client()

with open("/var/run/secrets/kubernetes.io/serviceaccount/namespace") as f:
    NAMESPACE = f.read()
NAMESPACE

## Component to be tested

In [ ]:
convert_model_to_onnx_comp = kfp.components.load_component_from_file("component.yaml")

## Test Data Generators

In [ ]:
def generate_test_model(model_dir: OutputPath(str)):
    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import Concatenate, Dense, Input
    import os
    from transformers import DefaultDataCollator

    from datasets import Dataset

    data = {
        "feature1": [[1, 2, 3, 4, 5], [2, 3, 4, 5, 6], [3, 4, 5, 6, 7]],
        "feature2": [True, False, True],
        "label1": [1, 0, 2],
        "label2": [True, False, True],
    }

    dataset = Dataset.from_dict(data)

    data_collator = DefaultDataCollator(return_tensors="tf")
    train_dataset = dataset.to_tf_dataset(
        columns=["feature1", "feature2"],
        label_cols=["label1", "label2"],
        shuffle=True,
        batch_size=32,
        collate_fn=data_collator,
    )

    input1 = Input(shape=(5,), name="feature1")
    input2 = Input(shape=(1,), name="feature2")
    input = Concatenate()([input1, input2])
    dense1 = Dense(units=32, activation="tanh", kernel_initializer="random_normal")(
        input
    )
    output1 = Dense(units=1, kernel_initializer="random_normal", name="label1")(dense1)
    output2 = Dense(units=1, kernel_initializer="random_normal", name="label2")(dense1)

    model = Model(inputs=[input1, input2], outputs=[output1, output2])

    model.compile(loss="mse", optimizer="sgd", metrics=["accuracy"])

    model.fit(train_dataset, epochs=10, batch_size=32)

    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model.save(model_dir)


generate_test_model_comp = kfp.components.create_component_from_func(
    func=generate_test_model, base_image=BASE_IMAGE
)

## Create pipeline

In [ ]:
@dsl.pipeline(
    name="Component Test - Convert Model to ONNX", description="A simple component test"
)
def train_pipeline():
    generate_test_model_task = generate_test_model_comp()

    convert_model_to_onnx_comp(model_dir=generate_test_model_task.outputs["model_dir"])

## Run the pipeline within an experiment

In [ ]:
KFP_CLIENT.create_run_from_pipeline_func(
    train_pipeline, arguments={}, namespace=NAMESPACE
)